In [1]:
import os
import json
import pandas as pd

In [2]:
PATH_TO_FILE = "/home/me/Projects/Python/coco-annotations-to-csv/annotations/person_keypoints_val2017.json"
with open(PATH_TO_FILE) as json_file:
    data = json.load(json_file)

In [3]:
# select all images out of given dataset
df_images = pd.DataFrame(data['images'])
df_images = df_images.set_index('id')
df_images.index.names = ['image_id'] #for merging readability later

In [4]:
# pull off image metadata
df_images_info =  pd.DataFrame(df_images[["file_name","width","height","coco_url"]])
df_images_info.columns = ["image_name","image_width","image_height","image_url"]

In [5]:
# all possible image descriptions eg. bounding boxes
df_annotations = pd.DataFrame(data['annotations'])
df_annotations = df_annotations.set_index('image_id')

In [6]:
# extract category_id out of each annotation for labeling later
df_annotations_category_id = pd.DataFrame(df_annotations["category_id"])

# done to keep image_id as primary key for relationship between images and annotations
df_annotations_category_id.reset_index(inplace=True)
df_annotations_category_id.set_index('category_id', inplace=True)

In [7]:
# extract all possible categories of objects present at image eg. persons
df_categories = pd.DataFrame(data['categories'], columns=["id","name"])
df_categories.set_index('id', inplace=True)

In [8]:
df_images_category_name = df_annotations_category_id.join(df_categories)
df_images_category_name.set_index('image_id', inplace=True)
df_images_category_name.columns = ["label"]

In [9]:
# pull off image border box coordinates
df_bbox = pd.DataFrame(df_annotations['bbox'].values.tolist(), columns=["x","y","width","height"], index=df_annotations.index)

In [10]:
# calculate missing bbox corners
df_xmax = pd.DataFrame(df_bbox["x"] + df_bbox["width"], columns = ["x_max"], index=df_annotations.index)
df_ymin = pd.DataFrame(df_bbox["y"] - df_bbox["height"], columns = ["y_min"],index=df_annotations.index)

In [11]:
# concatenate image corners location
df_coords = pd.concat([df_bbox["x"], df_ymin, df_xmax, df_bbox["y"]], axis=1)
df_coords.columns = ["x_min","y_min","x_max","y_max"]

In [12]:
# merge all dataframes to one
df_labeled_with_info = pd.merge(df_images_category_name, df_images_info, on=['image_id'])
df_to_export = pd.merge(df_labeled_with_info, df_coords, on=['image_id'])

# ensuring that columns order is as requested, you can safely change columns order below 
# !WARNING! may be computation expensive - could alsoe be done also by single poping/inserting - only image_url is not at position
final_columns_order = ["label","image_name","image_width","image_height","x_min","y_min","x_max","y_max","image_url"]
df_to_export = df_to_export.reindex(columns = final_columns_order, copy=False)
df_to_export.drop_duplicates(inplace=True)
df_to_export

,label,image_name,image_width,image_height,x_min,y_min,x_max,y_max,image_url
image_id,,,,,,,,,
425226,person,000000425226.jpg,480,640,73.35,-166.48,373.93,206.02,http://images.cocodataset.org/val2017/00000042...
440475,person,000000440475.jpg,640,427,491.91,110.11,526.76,183.51,http://images.cocodataset.org/val2017/00000044...
266400,person,000000266400.jpg,640,396,36.32,-53.27,74.88,0.59,http://images.cocodataset.org/val2017/00000026...
266400,person,000000266400.jpg,640,396,0.00,-137.52,15.09,0.99,http://images.cocodataset.org/val2017/00000026...
69213,person,000000069213.jpg,640,427,276.35,-390.53,521.04,9.60,http://images.cocodataset.org/val2017/00000006...
...,...,...,...,...,...,...,...,...,...
351096,person,000000351096.jpg,500,375,0.00,68.19,100.32,219.16,http://images.cocodataset.org/val2017/00000035...
2431,person,000000002431.jpg,457,640,102.61,-120.68,204.77,0.00,http://images.cocodataset.org/val2017/00000000...
2431,person,000000002431.jpg,457,640,0.00,-135.90,80.53,0.00,http://images.cocodataset.org/val2017/00000000...
